In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# Get the data from tsm and qcom with inteval of 30 minutes

msft = yf.Ticker("MSFT")
adbe = yf.Ticker("ADBE")

msft_data = msft.history(period="60d", interval="30m")
adbe_data = adbe.history(period="60d", interval="30m")

In [3]:
# plot both tickers on go figure

fig = go.Figure()
fig.add_trace(go.Scatter(x=msft_data.index, y=msft_data['Close'], name='TSM'))
fig.add_trace(go.Scatter(x=adbe_data.index, y=adbe_data['Close'], name='QCOM'))

fig.show()

In [4]:
print(msft_data)

                                 Open        High         Low       Close  \
Datetime                                                                    
2024-02-21 09:30:00-05:00  399.399994  400.040009  397.220001  398.809998   
2024-02-21 10:00:00-05:00  398.809998  400.470001  398.730011  399.410004   
2024-02-21 10:30:00-05:00  399.380005  400.010010  398.730011  399.790009   
2024-02-21 11:00:00-05:00  399.765991  400.709991  399.174011  399.809998   
2024-02-21 11:30:00-05:00  399.809998  400.149994  399.545013  400.019989   
...                               ...         ...         ...         ...   
2024-05-15 13:30:00-04:00  422.550995  423.059998  422.239990  422.954987   
2024-05-15 14:00:00-04:00  422.969910  423.670013  422.850006  423.540009   
2024-05-15 14:30:00-04:00  423.579987  423.589996  423.059998  423.260010   
2024-05-15 15:00:00-04:00  423.220001  423.809998  423.109985  423.410004   
2024-05-15 15:30:00-04:00  423.399994  423.570007  422.559998  423.149994   

In [5]:
# plot eith candle stick

fig = go.Figure(data=[go.Candlestick(x=msft_data.index,
                open=msft_data['Open'],
                high=msft_data['High'],
                low=msft_data['Low'],
                close=msft_data['Close'])])

# plot moving average of 20 ticks and 80 ticks

fig.add_trace(go.Scatter(x=msft_data.index, y=msft_data['Close'].rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=msft_data.index, y=msft_data['Close'].rolling(window=20).mean(), name='20 Ticks MA'))

fig.show()

In [6]:
# count the number of times the 5 ticks MA crosses the 20 ticks MA

msft_data['5T_MA'] = msft_data['Close'].rolling(window=5).mean()
msft_data['20T_MA'] = msft_data['Close'].rolling(window=20).mean()

msft_data['signal'] = np.where(msft_data['5T_MA'] > msft_data['20T_MA'], 1, 0)
msft_data['signal'] = msft_data['signal'].diff()


print(msft_data['signal'].value_counts())

# plot the buy and sell signals

fig = go.Figure(data=[go.Candlestick(x=msft_data.index,
                open=msft_data['Open'],
                high=msft_data['High'],
                low=msft_data['Low'],
                close=msft_data['Close'])])

fig.add_trace(go.Scatter(x=msft_data.index, y=msft_data['Close'].rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=msft_data.index, y=msft_data['Close'].rolling(window=20).mean(), name='20 Ticks MA'))

buy_signals = msft_data[msft_data['signal'] == 1]
sell_signals = msft_data[msft_data['signal'] == -1]

fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))

fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

fig.show()

signal
 0.0    732
 1.0     24
-1.0     23
Name: count, dtype: int64


In [7]:
# calculate profit and loss

msft_data['buy_price'] = np.where(msft_data['signal'] == 1, msft_data['Close'], np.nan)
msft_data['sell_price'] = np.where(msft_data['signal'] == -1, msft_data['Close'], np.nan)

msft_data['buy_price'] = msft_data['buy_price'].ffill()
msft_data['sell_price'] = msft_data['sell_price'].ffill()

msft_data['PnL'] = msft_data['sell_price'] - msft_data['buy_price']

print(msft_data['PnL'].sum())

# plot the profit and loss

fig = go.Figure(data=[go.Candlestick(x=msft_data.index,
                open=msft_data['Open'],
                high=msft_data['High'],
                low=msft_data['Low'],
                close=msft_data['Close'])])

fig.add_trace(go.Scatter(x=msft_data.index, y=msft_data['Close'].rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=msft_data.index, y=msft_data['Close'].rolling(window=20).mean(), name='20 Ticks MA'))

buy_signals = msft_data[msft_data['signal'] == 1]
sell_signals = msft_data[msft_data['signal'] == -1]

fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

fig.show()

-449.3616638183594


In [8]:
ratio_0 = (msft_data['Close'].iloc[0]) / (adbe_data['Close'].iloc[0])
ratio_0

0.7392214968648633

In [9]:
adbe_data['Close_NORM'] = adbe_data['Close'].multiply(ratio_0).round(6)

In [10]:
msft_adbe_dif = msft_data['Close'] - adbe_data['Close_NORM']
msft_adbe_dif

Datetime
2024-02-21 09:30:00-05:00   -4.414063e-07
2024-02-21 10:00:00-05:00    1.175403e+00
2024-02-21 10:30:00-05:00    3.604239e+00
2024-02-21 11:00:00-05:00    3.646426e+00
2024-02-21 11:30:00-05:00    3.771414e+00
                                 ...     
2024-05-15 13:30:00-04:00    6.208922e+01
2024-05-15 14:00:00-04:00    6.255229e+01
2024-05-15 14:30:00-04:00    6.218726e+01
2024-05-15 15:00:00-04:00    6.333521e+01
2024-05-15 15:30:00-04:00    6.434666e+01
Length: 780, dtype: float64

In [11]:
# Plot the difference between the two tickers

fig = go.Figure()
fig.add_trace(go.Scatter(x=msft_adbe_dif.index, y=msft_adbe_dif, name='TSM-QCOM'))

fig.show()

In [12]:
# Plot the two normalized tickers

fig = go.Figure()
fig.add_trace(go.Scatter(x=msft_data.index, y=msft_data['Close'], name='TSM'))
fig.add_trace(go.Scatter(x=adbe_data.index, y=adbe_data['Close_NORM'], name='QCOM'))

fig.show()

In [13]:
# Plot the difference and its moving average

fig = go.Figure()
fig.add_trace(go.Scatter(x=msft_adbe_dif.index, y=msft_adbe_dif, name='TSM-QCOM'))
fig.add_trace(go.Scatter(x=msft_adbe_dif.index, y=msft_adbe_dif.rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=msft_adbe_dif.index, y=msft_adbe_dif.rolling(window=20).mean(), name='20 Ticks MA'))

fig.show()


In [14]:
msft_adbe_dif = msft_adbe_dif.to_frame()
msft_adbe_dif.columns = ['Close']

In [20]:
msft_adbe_dif['10T_MA'] = msft_adbe_dif['Close'].rolling(window=10).mean()
msft_adbe_dif['60T_MA'] = msft_adbe_dif['Close'].rolling(window=60).mean()

msft_adbe_dif['signal'] = np.where(msft_adbe_dif['10T_MA'] > msft_adbe_dif['60T_MA'], 1, 0)
msft_adbe_dif['signal'] = msft_adbe_dif['signal'].diff()

print(msft_adbe_dif['signal'].value_counts())

# Plot the signals

fig = go.Figure()
fig.add_trace(go.Scatter(x=msft_adbe_dif.index, y=msft_adbe_dif['Close'], name='TSM-QCOM'))
fig.add_trace(go.Scatter(x=msft_adbe_dif.index, y=msft_adbe_dif.rolling(window=10).mean()['Close'], name='10 Ticks MA'))
fig.add_trace(go.Scatter(x=msft_adbe_dif.index, y=msft_adbe_dif.rolling(window=50).mean()['Close'], name='50 Ticks MA'))

buy_signals = msft_adbe_dif[msft_adbe_dif['signal'] == 1]
sell_signals = msft_adbe_dif[msft_adbe_dif['signal'] == -1]

fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

 # -1 buy || 1 sell

msft_data['signal'] = -msft_adbe_dif['signal']
adbe_data['signal'] = msft_adbe_dif['signal'] # opposite signal

fig.show()

# plot the profit and loss

msft_data['buy_price'] = np.where(msft_data['signal'] == 1, msft_data['Close'], np.nan)
msft_data['sell_price'] = np.where(msft_data['signal'] == -1, msft_data['Close'], np.nan)

adbe_data['buy_price'] = np.where(adbe_data['signal'] == 1, adbe_data['Close_NORM'], np.nan)
adbe_data['sell_price'] = np.where(adbe_data['signal'] == -1, adbe_data['Close_NORM'], np.nan)

tsm_profit = msft_data['sell_price'].sum() - msft_data['buy_price'].sum()
qcom_profit = adbe_data['sell_price'].sum() - adbe_data['buy_price'].sum()

print('TSMC Profit => ', tsm_profit)
print('QCOM Profit => ', qcom_profit)

pair_tsm_qcom_profit = tsm_profit + qcom_profit

print('Pair profit => ', pair_tsm_qcom_profit)


signal
 0.0    764
 1.0      8
-1.0      7
Name: count, dtype: int64


TSMC Profit =>  437.8797607421875
QCOM Profit =>  -429.533868
Pair profit =>  8.345892742187516


In [16]:
#Consider TSM as the first and QCOM as the second ticker.
#When there is a buy signal for TSM, we buy TSM and sell QCOM.
#When there is a sell signal for TSM, we sell TSM and buy QCOM.

In [17]:
#Copy signal column from difference dataframe to TSM dataframe
msft_data['signal'] = msft_adbe_dif['signal']
adbe_data['signal'] = -msft_adbe_dif['signal']

In [18]:
# calculate cointegration between the two tickers
from statsmodels.tsa.stattools import coint

cointegration = coint(msft_data['Close'], adbe_data['Close_NORM'])
print(cointegration)



(-2.4792663977685003, 0.28828893937823313, array([-3.91055417, -3.34398476, -3.0498989 ]))
